## Задание

В задании требовалось обработать коллекцию страниц в домене первого уровня .by BY.WEB. Требовалось проанализировать следующие характеристики (далее в тексте коллекция — все слова из документов, словарь — уникальные слова из документов):



 - Общие

   - Общее количество документов
    
 - Текстовые
 
   - Распределение длин документов в словах и байтах
   - Средняя длина документа в словах и байтах
   - Соотношение объёма документа и исходной HTML-страницы
   
 - Морфологические
 
   - Доля стоп-слов в коллекции
   - Доля слов латиницей в коллекции и словаре
   - Средняя длина слова в коллекции и словаре
   - Частота в коллекции для слов из словаря
   - Инвертированная документная частота для слов из словаря

Также требовалось построить график зависимости ранга слова от частоты в логарифмических координатах и граф гиперссылок между страницами коллекции.


## Описание данных

На вход было подано 10 файлов в формате xml, размер каждого из которых составлял около 1 Гб. Каждый файл содержал 20000 документов: содержимого документа, ссылки на документ и целочисленного идентификатора. Содержимое и ссылка были закодированы в base64, раскодированный текст кодирован в cp1251.

## Описание методов, инструментов, подходов. Описание эксперимента

Код для обработки и анализа коллекции был написан на языке Python.

Для извлечения данных из xml использовалась библиотека `lxml` и её метод `iterparse`, поскольку это достаточно быстрый и простой способ обработки документа. Для декодирования из base64 использовалась библиотека `pybase64`, поскольку она быстрее стандартной `base64`. Для разбора HTML-страниц использовалась библиотека `Beautiful Soup` с парсером `lxml`, при этом из страницы вырезались комментарии и содержимое с тэгами `script` и `style`, содержащими код на языке Javascript или CSS-фрагменты. 

После разбора HTML-страницы получался текстовый документ, который затем разбивался на лемматизированные слова при помощи библиотеки `pymystem3`. Из полученных слов выбирались те, которые состояли букв и цифр, чтобы избежать знаков препинания и прочих символов, после чего слова приводились в нижний регистр (`mystem` обрабатывает только слова русского языка, но в тексте также встречалось некоторое количество слов латиницей).

Полный текст для каждого документа не сохранялся. Вместо этого учитывалась статистика по документу: размер документа в словах и байтах, соотношение объёма текста и исходного HTML-документа, количество слов в документе, а также ссылки, содержащиеся в документе. Для каждого документа найденные слова записывались в словарь (если слово уже присутствовало в словаре, для него инкрементировались значения частоты в коллекции и документной частоты). 

По собранной статистике были вычислены следующие характеристики: общее количество документов, распределение длин документов в словах и байтах, средний размер документа в словах и байтах, среднее соотношение объёма текста и исходного HTML-документа. Для визуализации была выбрана библиотека `plotly`, позволяющая строить интерактивные гистограммы и графики.

По словарю были вычислены следующие характеристики: средняя длина слова в коллекции и словаре, доля стоп-слов в коллекции (стоп-слова были взяты из архива `stopwords`, используемого в `nltk`), доля слов латиницей в коллекции и словаре, частота в коллекции и инвертированная документная частота. Были определены пять самых больших значений частоты в коллекции и пять самых маленьких значений инвертированной документной частоты и выведены слова, соответствующие полученным значениям. Стоп-слова при этом не учитывались. Инвертированная документная частота слова $w$ вычислялась по формуле $\log \frac{N}{df_w}$, где $N$ — общее количество документов, $df_w$ — документная частота слова $w$. 

Для построения графика зависимости ранга слова от частоты в логарифмических координатах словарь был отсортирован по частоте в коллекции. 


Для работы с гиперссылками использовалась библиотека `urllib`, позволившая удалить из ссылок параметры, а также перевести относительные ссылки в абсолютные, используя адрес документа. Для визуализации с помощью библиотеки `pyvis` по обработанным ссылкам строился граф. Размер вершины, соответствующей документу, зависит от количество ссылок на него. Для большей наглядности в граф попали документы, на которые ссылаются не менее 300 прочих документов.


## Результаты

In [ ]:
import lxml.etree as et

from dictionary import *
from utils import *
from document import *
from graph import *


def parse_xml(filename, docs_stat):
    context = et.iterparse(filename, tag='document')

    for (_, elem) in context:
        content = elem[0].text
        url = elem[1].text
        doc_id = int(elem[2].text)
        elem.clear()

        try:
            doc = Document(decode_base64_cp1251(content), doc_id, decode_base64_cp1251(url))
            stats = doc.calc_doc_stats()
            docs_stat.append(stats)
            graph.add_document(stats)
            dictionary.add_doc_words(doc.words)
        except:
            print("Unable to parse " + str(doc_id))


XML_FOLDER = "byweb_for_course"


docs_stat = []
mystem = Mystem()
dictionary = Dictionary()
graph = LinkGraph()

for filename in os.listdir(XML_FOLDER):
    if filename.endswith(".xml"):
        parse_xml(XML_FOLDER + os.sep + filename, docs_stat)

In [ ]:
print("Total documents count: " + str(len(docs_stat)))
print("Average document length: " + float_to_str(average_doc_size_in_words(docs_stat)) + " words")
print("Average document length: " + float_to_str(average_doc_size_in_bytes(docs_stat)) + " bytes")
print("Average text content to HTML content ratio: " + float_to_str(average_doc_text_to_html_ratio(docs_stat)))
plot_histogram(
    data=[stat.words_cnt for stat in docs_stat],
    title="Length in words distribution",
    x_axis_title="words",
    y_axis_title="documents",
    step=250
)
plot_histogram(
    data=[stat.bytes_cnt for stat in docs_stat],
    title="Length in bytes distribution",
    x_axis_title="bytes",
    y_axis_title="documents",
    step=5000
)

print("Collection stop words ratio: " + float_to_str(dictionary.stop_words_proportion(in_collection=True)))
print("Collection latin words ratio: " + float_to_str(dictionary.latin_words_proportion(in_collection=True)))
print("Dictionary latin words ratio: " + float_to_str(dictionary.latin_words_proportion(in_collection=False)))
print("Dictionary average word length: " + float_to_str(dictionary.average_dic_word_len()))
print("Collection average word length: " + float_to_str(dictionary.average_word_len()))

print("Words with largest collection frequency: " +
      str(dictionary.most_popular_word(lambda v: dictionary.dict[v].cnt)))
print("Words with smallest inverse document frequency: " +
      str(dictionary.most_popular_word(lambda v: math.log10(len(docs_stat) / dictionary.dict[v].doc_cnt),
                                       get_max=False)))

rf = rank_frequency({word: value.cnt for word, value in dictionary.dict.items()})
plot_line(rf, "Rank-frequency", "rank", "frequency")

In [ ]:
graph.show()

## Анализ

В коллекции преобладают небольшие документы длиной до 1000 слов (средняя длина документа — 839 слов). 

Средняя длина документа в коллекции получилась короче, чем в словаре, поскольку в коллекции достаточно большой (20%) процент стоп-слов, которые в массе своей довольно короткие.

Текст, видимый посетителю сайта, занимает лишь малую часть документа (19%) в то время, как основной объем приходится на разметку, CSS и JS.

Слов латиницей получилось неожиданно много — возможно, потому, что почти на каждой странице встречается несколько латинских слов (название сайта, ссылки и т.п.), а на некоторых довольно много (страницы с форума linux.by). 

График зависимости логарифма частоты слова от логарифма его ранга подтверждает закон Ципфа: каждое следующее по частоте слово встречается резко реже предыдущего.

Среди наиболее часто встречающихся в документах (т.е. с наименьшей инвертированной документной частотой) слов находятся в основном такие, которые как-то описывают функциональность сайта (“поиск”, “главный” в смысле “главная страница”), также естественным образом оказалось популярно слово “беларусь”. 

Частыми оказались слова “url”, “org”, “www”,  “http”.  Вероятно, из-за того, что нередко ссылки оставляют простым текстом.  Также встречается слово “г”, что скорее всего является сокращением от “год”.

Как ожидалось, страницы одного домена часто ссылаются друг на друга и образуют кластеры. Оказалось очень много ссылок на tut.by, в частности на catalog.tut.by/. Кроме этого можно отметить news.br.by/, cards.br.by/, photoclub.by, avto.tdj.by.
